In [23]:
from dash import Dash, html, dcc, Input, Output, callback, Patch, ALL, State

import dash_cytoscape as cyto
from matplotlib import style


#legacy function
def addArcToGraph(arc, graph):
    if arc in graph:
        return graph
    elif graph == "":
        return graph + arc
    else:
        return graph + ";;" + arc

app = Dash()

#Get the ID for the next node to be added to the list
#Goes through all ids and if they are integer finds the highest value and adds 1 to it, if none are integers returns 0. 
def getNextNodeID(elements):
    idList = []
    #print(elements)
    for element in elements:
        #print(element, ' element')
        if 'id' in element['data'].keys():
            if (element['data']['id'].isdigit()):
                idList.append(int(element['data']['id']))
    if len(idList) == 0:
        return 1
    else:
        return max(idList) + 1   
    
def addNewNode(elements):
    nodeDict = {}
    dataDict = {}
    nodeDict['id'] = str(getNextNodeID(elements))
    nodeDict['label'] = str(getNextNodeID(elements))
    dataDict['data'] = nodeDict
    dataDict['classes'] = 'default'
    #print(elements, ' addNode ', elements.append(dataDict))
    elements.append(dataDict)
    return elements

def removeNode(node, elements):
    #print('before remove ', elements)
    for element in elements:
      if 'id' in element['data'].keys():
        if node['id'] == element['data']['id']:
            elements.remove(element)
    #print('after remove ', elements)
    return elements

def removeEdge(edge, elements):
    for element in elements:
        if 'source' in element['data'].keys():
            if (element['data']['source'] == edge['source']) and (element['data']['target'] == edge['target']):
                elements.remove(element)
    return elements

def isEdgeInElements(edge, elements):
    for element in elements:
        if 'source' in element['data'].keys():
            if (element['data']['source'] == edge['source']) and (element['data']['target'] == edge['target']):
                return True
    return False

def addNewEdge(node1, node2, elements):
    edgeDict = {}
    dataDict = {}
    edgeDict['source'] = node1['id']
    edgeDict['target'] = node2['id']
    dataDict['data'] = edgeDict
    if isEdgeInElements(edgeDict, elements):
        return elements
    else:
        elements.append(dataDict)
        return elements

def setNodeColorToselected(node, elements):
    for element in elements:
        if 'id' in element['data'].keys():
            if element['data']['id'] == node['id']:
                element['classes'] = 'selected'
    return elements

def unselectNode(node, elements):
    for element in elements:
      if 'id' in element['data'].keys():
        if element['data']['id'] == node['id']:
            if 'classes' in element.keys():
                element['classes'] = 'default'
    return elements

app.layout = [
    dcc.Store(id='CurrentlySelectedNode'),
    dcc.Store(id='selectedEdge'),
    dcc.Store(id='memory-output'),
    dcc.Store(id='memory-output1'),
    #graph entry
    html.H3(id="GraphHeading", children="Current Graph"),
    html.H5(id="graph", children=""),

    dcc.RadioItems(
        id="radioSelection",
        options={
            'sequence': 'Sequence Input',
            'graph': 'Graph Edge Input',
        },
        value='sequence'
    ),
    html.Button(id='addNode_',children='Add a new Graph Node'),
    html.Button(id='RemoveNode', children='Remove a Specific Graph Node'),
    html.Button(id='RemoveEdge', children='Remove a Specific Graph Edge'),
    dcc.Store(id='Flags'),
    dcc.Store(id='edgeFlags'),
    html.H5(id="cytoText"),
    cyto.Cytoscape(
        id='cytoscape',
        elements=[
            {'data': {'id': 'escape', 'label': 'escape'}},
            {'data': {'id': '1', 'label': '1'}}, 
            {'data': {'id': '2', 'label': '2'}}, 
            {'data': {'id': '3', 'label': '3'}},
            {'data': {'source': '1', 'target': '2'}}, 
            {'data': {'source': '2', 'target': '3'}}
        ],
        #layout={'name': 'breadthfirst'},
        style={'width': '400px', 'height': '500px'},
        stylesheet=[
            {
                'selector': 'default',
                'style': {
                    'background-color': '#969696',
                    'content': 'data(label)'
                }
            },
            {
                'selector': '.default',
                'style': {
                    'background-color': '#969696',
                    'content': 'data(label)'
                }
            },
            {
                'selector': '.selected',
                'style': {
                    'background-color': '#0099ff',
                    'content': 'data(label)'
                }
            }
        ]
    ),



    html.Div(children=[
        dcc.Input(id="firstNode", value="", type="number", style={'display': 'inline-block'}), 
        dcc.RadioItems(['nseq', 'nor', 'nand', 'nxor'], value='nseq', id='firstType', style={'display': 'inline-block'}),
        dcc.Input(id="secondNode", value="", type="number", style={'display': 'inline-block'}),
        dcc.RadioItems(['nseq', 'nor', 'nand', 'nxor'], value='nseq', id='secondType', style={'display': 'inline-block'}),
        html.Button(id='add_arc', n_clicks=0, children='Add Graph Arc'),
        html.Button(id='remove_arc', n_clicks=0, children='Remove Graph Arc')
        ]),
    
    #sequence entry
    html.H1(id="EmptySpace", children=""),
    html.H1(id="EmptySpace2", children=""),
    html.H3(id="SequenceHeading", children="Current Sequence"),
    html.Div([
        html.H5(id="startingAction", children="S", style={'display': 'inline-block'}),
        html.H5(id="sequence", children="", style={'display': 'inline-block'}),
        html.H5(id="endingAction", children=",F", style={'display': 'inline-block'}),
        html.H5(id="storedData"),
        html.H5(id="storedData1"),
        ]),
    html.Div(children=[
        dcc.Input(id="inputAction", value="", type="number", style={'display': 'inline-block'}), 
        html.Button(id='add_action', n_clicks=0, children='Add Action'),
        html.Button(id='store_data', n_clicks=0, children='Store Data'),
        html.Button(id='show_data', n_clicks=0, children='Show Data'),
        ])
]

@callback(
   Output(component_id="graph", component_property="children", allow_duplicate=True),
   State(component_id="graph", component_property="children"),
   Input(component_id='add_arc', component_property='n_clicks'),
   Input(component_id='remove_arc', component_property='n_clicks'),
   State(component_id="firstNode", component_property="value"),
   State(component_id="firstType", component_property="value"),
   State(component_id="secondNode", component_property="value"),
   State(component_id="secondType", component_property="value"),
   prevent_initial_call=True,
   allow_duplicate=True
)
def update_graph(graph, addArc, remArc, firstN, firstT, secondN, secondT):
   arc = "(n[" + str(firstN) + "]: " + str(firstT) + ")->(n[" + str(secondN) + "]: " + str(secondT) + ")"
   return addArcToGraph(arc, graph)

@callback(
   Output(component_id="sequence", component_property="children", allow_duplicate=True),
   State(component_id="sequence", component_property="children"),
   Input(component_id='add_action', component_property='n_clicks'),
   State(component_id="inputAction", component_property="value"),
   prevent_initial_call=True,
   allow_duplicate=True
)
def update_sequence(sequence, add_sequence_button, inputAction):
   return (sequence + "," + str(inputAction))

@callback(
    Output(component_id='sequence', component_property='children', allow_duplicate=True),
    Output(component_id='graph', component_property='children', allow_duplicate=True),
    Input(component_id='cytoscape', component_property='tapNodeData'),
    State(component_id="radioSelection", component_property='value'),
    State(component_id="sequence", component_property="children"),
    State(component_id='graph', component_property='children'),
    prevent_initial_call=True,
    allow_duplicate=True
)
def get_info_from_cyto(cytoscape, radio, sequence, graph):
    #print(cytoscape['label'], radio)
    if radio=='sequence':
        return (sequence + "," + str(cytoscape['label'])), graph
    elif radio=='graph':
        arc = "(n[" + str(cytoscape['label']) + "]: " + 'nseq' + ")->(n[" + str(cytoscape['label']) + "]: " + 'nseq' + ")"
        return sequence, addArcToGraph(arc, graph)
    else:
        return sequence, graph
    
@callback(
    Output("memory-output", "data"),
    Output("memory-output1", "data"),
    #State("sequence","children"),
    State("cytoscape", "elements"),
    Input("store_data","n_clicks"),
    prevent_initial_call=True,
)
def storeData(cytoscape , clicks):
    return (cytoscape), cytoscape[0]['data']['id']

@callback(
    Output("storedData", "children"),
    Output("storedData1", "children"),
    Input("show_data","n_clicks"),
    State("memory-output","data"),
    State("memory-output1", "data"),
    prevent_initial_call=True,
)
def showData(clicks,memor1y, memory2):
    return getNextNodeID((addNewNode(addNewNode(memor1y)))), memory2
    
@callback(
    Output("cytoscape", "elements",  allow_duplicate=True),
    State("cytoscape", "elements"),
    Input("addNode_", "n_clicks"),
    prevent_initial_call=True,
)
def addNewNodeCallback(cyto, n_cl):
    return addNewNode(cyto)

@callback(
    Output("Flags", "data", allow_duplicate=True),
    Output("RemoveNode", "children", allow_duplicate=True),
    Output("cytoscape", "elements", allow_duplicate=True),
    Output("CurrentlySelectedNode", "data"),
    State("CurrentlySelectedNode", "data"),
    State("cytoscape", "elements"),
    State("RemoveNode", "children"),
    Input("cytoscape", "tapNodeData"),
    State("Flags", "data"),
    prevent_initial_call=True,
)
def tapNodeAddEdge(prevNode, elements, rmNodeButton, tappedNode, flags):
    #print(elements)
    store = None
    #print(flags)
    if (flags != None) and ('removeNode' in flags.keys()) and (flags['removeNode'] == True):
            elements = removeNode(tappedNode, elements)
            flags['removeNode'] = False
            rmNodeButton = "Remove a Specific Graph Node"
            return flags, rmNodeButton, elements, store
    else:
        if prevNode == None:
            store = tappedNode
            elements = setNodeColorToselected(tappedNode, elements)
            rmNodeButton = "Remove the Selected Graph Node"
        else:
            elements = unselectNode(prevNode, elements)
            elements = addNewEdge(prevNode, tappedNode, elements)
            elements = unselectNode(tappedNode, elements)
            store = None   
            rmNodeButton = "Remove a Specific Graph Node"
    return flags, rmNodeButton, elements, store


@callback(
    Output("cytoscape", "elements", allow_duplicate=True),
    Output("Flags", "data", allow_duplicate=True),
    Output("RemoveNode", "children", allow_duplicate=True),
    Output("CurrentlySelectedNode", "data", allow_duplicate=True),
    State("cytoscape", "elements"),
    State("Flags", "data"),
    State("CurrentlySelectedNode", "data"),
    Input("RemoveNode", "n_clicks"),
    State("RemoveNode", "children"),
    prevent_initial_call=True,
    )
def removeNodePressed(elements, flags, selectedNode, clicks, rmNodeButton):
    if selectedNode != None:
        elements = removeNode(selectedNode, elements)
        rmNodeButton = "Remove a Specific Graph Node"
        selectedNode = None
    else:
        dictF = {}
        dictF['removeNode'] = True
        flags = dictF
        rmNodeButton = "Select Graph Node To Remove"
    return elements, flags, rmNodeButton, selectedNode


@callback(
    Output("RemoveEdge", "children", allow_duplicate=True),
    Output("cytoscape", "elements", allow_duplicate=True),
    Output("edgeFlags", "data", allow_duplicate=True),
    Output("selectedEdge", "data", allow_duplicate=True),
    Input("RemoveEdge", "n_clicks"),
    State("edgeFlags", "data"),
    State("cytoscape", "elements"),
    State("selectedEdge", "data"),
    prevent_initial_call=True,
)
def removeEdgeButton(button, edgeFlags, elements, selectedEdge):
    if (edgeFlags == None) or (edgeFlags == False):
        if (selectedEdge == None):
            flag = True
            edgeFlags = flag
            buttonMSG = "Select Edge to be Removed"
        else:
            elements = removeEdge(selectedEdge, elements)
            selectedEdge = None
            buttonMSG = "Remove a Specific Graph Edge"
    else:
        edgeFlags = False
        buttonMSG = "Remove a Specific Graph Edge"
    return buttonMSG, elements, edgeFlags, selectedEdge


@callback(
    Output("RemoveEdge", "children", allow_duplicate=True),
    Output("cytoscape", "elements", allow_duplicate=True),
    Output("edgeFlags", "data", allow_duplicate=True),
    Output("selectedEdge", "data", allow_duplicate=True),
    State("edgeFlags", "data"),
    State("cytoscape", "elements"),
    Input("cytoscape", "tapEdgeData"),
    prevent_initial_call=True,
)
def selctedEdge(flags, elements, edge):
    selectedEdge = None
    if (flags != None) and (flags == True):
        elements = removeEdge(edge, elements)
        selectedEdge = None
        buttonMSG = "Remove a Specific Graph Edge"
        flags = False
    if (flags == None) or (flags == False):
        selectedEdge = edge
        buttonMSG = "Remove Selected Edge"
    return buttonMSG, elements, flags, selectedEdge

    
        
app.run(debug=True)
#app.run(debug=True, host= "192.168.0.102", port=8052)
